In [4]:
import numpy as np
from quaos.paulis import PauliSum, PauliString
from quaos.gates import GateOperation, Circuit, Hadamard as H, SUM as CX, PHASE as S
from quaos.hamiltonian import *
from quaos.core.prime_Functions_Andrew import int_to_bases
from collections import defaultdict
import sympy as sym
from sympy.physics.quantum import TensorProduct,Operator

In [5]:
def modulo_2(expr):
    """
    Takes a SymPy expression and reduces its coefficients modulo 2.
    """
    # Expand the expression to handle all terms
    expr = expr.expand()
    
    # Iterate through the terms and apply modulo 2 to coefficients
    terms = expr.as_ordered_terms()
    mod_expr = sum(sym.Mod(term.as_coeff_Mul()[0], 2) * term.as_coeff_Mul()[1] for term in terms)
    
    return mod_expr

def reduce_exponents(expr):
    """
    Reduces all exponents in a SymPy expression to zero, assuming symbols are binary (0 or 1).
    
    Args:
        expr (sympy.Expr): The input SymPy expression.
    
    Returns:
        sympy.Expr: The modified expression with all exponents set to zero.
    """
    expr = sym.expand(expr)  # Expand the expression to handle all terms
    return expr.replace(lambda x: x.is_Pow, lambda x: x.base)

def symplectic_effect(circuit,phases = True):
    n_qudits = len(circuit.dimensions)
    #print(n_qudits)
    r_now = list(sym.symbols([f'r{i}' for i in range(1, n_qudits+1)]))
    omega = sym.symbols('omega')
    #print(len(r_now))
    s_now = list(sym.symbols([f's{i}' for i in range(1, n_qudits+1)]))
    r_next = [r_now[i] for i in range(n_qudits)]
    s_next = [s_now[i] for i in range(n_qudits)]

    #r1_start, r2_start, s1_start, s2_start = sym.symbols('r1 r2 s1 s2')
    X = Operator('X')
    Z = Operator('Z')

    phase = 0
    gates = circuit.gates
    qubits = circuit.indexes
    for i,g in enumerate(gates):
        if g.name == 'SUM':
            r_next[qubits[i][1]] = r_now[qubits[i][0]] + r_now[qubits[i][1]]
            s_next[qubits[i][0]] = s_now[qubits[i][0]] + s_now[qubits[i][1]]
            #phase += r_now[qubits[i][1]] * s_now[qubits[i][1]] * (r_now[qubits[i][0]] + s_now[qubits[i][0]] + 1)
        elif g.name == 'H' or g.name == 'HADAMARD':
            r_next[qubits[i][0]] = s_now[qubits[i][0]]
            s_next[qubits[i][0]] = r_now[qubits[i][0]]
            phase += s_now[qubits[i][0]] * r_now[qubits[i][0]]
        elif g.name == 'S' or g.name == 'PHASE':
            s_next[qubits[i][0]] = s_now[qubits[i][0]] + r_now[qubits[i][0]]
            #phase += s_now[qubits[i][0]] * r_now[qubits[i][0]]
        r_now = [r_next[i] for i in range(n_qudits)]
        s_now = [s_next[i] for i in range(n_qudits)]
    final = TensorProduct(X**(modulo_2(r_now[0])) * Z**(modulo_2(s_now[0])), X**(modulo_2(r_now[1])) * Z**(modulo_2(s_now[1])))
    for i in range(2, n_qudits):
        final = TensorProduct(final, X**(modulo_2(r_now[i])) * Z**(modulo_2(s_now[i])))
    
    if phases:
        display(omega**modulo_2(reduce_exponents(modulo_2(sym.simplify(phase)))) * final)
    else:
        display(final)
    return()

In [6]:
# Identity

list_add_id = []

# I: S
# X: S
# Z: S
# Y: S

'''
I  ->  I
X  ->  Z
Z  ->  Y
Y  ->  X
'''
list_add_id.append(Circuit(dimensions=[2,2], gates=[H(1,2),S(1,2)]))
list_add_id.append(Circuit(dimensions=[2,2], gates=[S(0,2),H(1,2),S(1,2),S(0,2)]))
list_add_id.append(Circuit(dimensions=[2,2], gates=[H(1,2),CX(1,0,2),S(1,2),CX(1,0,2)]))

'''
I  ->  I
X  ->  Y
Z  ->  Z
Y  ->  X
'''
list_add_id.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),CX(1,0,2),S(0,2),CX(1,0,2),CX(0,1,2)]))

'''
I  ->  I
X  ->  X
Z  ->  Y
Y  ->  Z
'''
list_add_id.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),H(1,2),S(1,2),H(1,2),CX(0,1,2)]))

# I: D
# X: D
# Z: S
# Y: S
'''
I  ->  I
X  ->  X
Z  ->  Y
Y  ->  Z
'''
list_add_id.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),S(1,2),H(1,2),S(1,2),CX(0,1,2)]))
list_add_id.append(Circuit(dimensions=[2,2], gates=[CX(1,0,2),CX(0,1,2),S(0,2),H(0,2),CX(0,1,2),S(0,2),CX(1,0,2)]))


# I: D
# X: S
# Z: D
# Y: S
'''
I  ->  I
X  ->  Y
Z  ->  Z
Y  ->  X
'''
list_add_id.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),S(0,2),S(1,2),CX(0,1,2),H(0,2),CX(1,0,2)]))


# I: S
# X: D
# Z: S
# Y: D
'''
I  ->  I
X  ->  Y
Z  ->  X
Y  ->  Z
'''
list_add_id.append(Circuit(dimensions=[2,2], gates=[CX(1,0,2),S(0,2),CX(1,0,2),H(1,2),CX(0,1,2),S(0,2)]))



In [9]:
start_paulis = [PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[0,0],standardise=False),
               PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[0,0],standardise=False),
               PauliSum(['x1z0 x0z1', 'x0z1 x0z1'], dimensions=[2, 2], phases=[0,0],standardise=False),
               PauliSum(['x1z0 x1z1', 'x0z1 x1z1'], dimensions=[2, 2], phases=[0,0],standardise=False)]

modes = ['SSSS','DDSS','DSDS','DSSD','SDDS','SDSD','SSDD','DDDD'] 
options = [0,1,2,3,4,5,6,7]
gate_options = [[],[],[],[],[],[],[],[]]

for mode in modes:
    for i,ci in enumerate(list_add_id):
        c = Circuit(dimensions=[2,2])
        id_c = add_phase(start_paulis[0],0,1,mode)
        for gj in ci.gates:
            c.add_gate(gj)
        for gj in id_c.gates:
            c.add_gate(gj)
        #print(c)
        #symplectic_effect(c,phases=False)
        '''
        for ip,p in enumerate(start_paulis):
            P_temp = c.act(p)
            if ip == 0:
                ini_s = 'I'
            elif ip == 1:
                ini_s = 'X'
            elif ip == 2:
                ini_s = 'Z'
            elif ip == 3:
                ini_s = 'Y'

            if P_temp.x_exp[0,1] == 0:
                if P_temp.z_exp[0,1] == 0:
                    print(ini_s,' -> ','I')
                else:
                    print(ini_s,' -> ','Z')
            elif P_temp.x_exp[0,1] == 1:
                if P_temp.z_exp[0,1] == 0:
                    print(ini_s,' -> ','X')
                else:
                    print(ini_s,' -> ','Y')

            phases = P_temp.phases
            if phases[0] == phases[1]:
                print('Same Phase')
            else:
                print('Different Phase')
        print()
        print()
        print()
        '''

        p_counter = 0
        p_ss = ''
        ini_ss = []
        for ip,p in enumerate(start_paulis):
            P_temp = c.act(p)
            if ip == 0:
                ini_s = 'I'
            elif ip == 1:
                ini_s = 'X'
            elif ip == 2:
                ini_s = 'Z'
            elif ip == 3:
                ini_s = 'Y'

            if P_temp.x_exp[0,1] == 0:
                if P_temp.z_exp[0,1] == 0:
                    ini_s += '-> I'
                    #print(ini_s,' -> ','I')
                    pass
                else:
                    ini_s += '-> Z'
                    #print(ini_s,' -> ','Z')
                    pass
            elif P_temp.x_exp[0,1] == 1:
                if P_temp.z_exp[0,1] == 0:
                    ini_s += '-> X'
                    #print(ini_s,' -> ','X')
                    pass
                else:
                    ini_s += '-> Y'
                    #print(ini_s,' -> ','Y')
                    pass
            ini_ss.append(ini_s)
            phases = P_temp.phases
            if phases[0] == phases[1]:
                #print('Same Phase')
                p_ss += 'S'
                p_counter += 1
            else:
                p_ss += 'D'
                #print('Different Phase')
                pass
        if p_ss == 'SSSS' and 0 in options:
            #print('Option 0: SSSS')
            #print(c)
            #print()
            #options.remove(0)
            gate_options[0].append(c)
        elif p_ss == 'DDSS' and 1 in options:
            #print('Option 1: DDSS')
            #print(c)
            #print()
            #options.remove(1)
            gate_options[1].append(c)
        elif p_ss == 'DSDS' and 2 in options:
            #print('Option 2: DSDS')
            #print(c)
            #print()
            #options.remove(2)
            gate_options[2].append(c)
        elif p_ss == 'DSSD' and 3 in options:
            #print('Option 3: DSSD')
            #print(c)
            #print()
            #options.remove(3)
            gate_options[3].append(c)
        elif p_ss == 'SDDS' and 4 in options:
            #print('Option 4: SDDS')
            #print(c)
            #print()
            #options.remove(4)
            gate_options[4].append(c)
        elif p_ss == 'SDSD' and 5 in options:
            #print('Option 5: SDSD')
            #print(c)
            #print()
            #options.remove(5)
            gate_options[5].append(c)
        elif p_ss == 'SSDD' and 6 in options:
            #print('Option 6: SSDD')
            #print(c)
            #print()
            #options.remove(6)
            gate_options[6].append(c)
        elif p_ss == 'DDDD' and 7 in options:
            #print('Option 7: DDDD')
            #print(c)
            #print()
            #options.remove(7)
            gate_options[7].append(c)

        if p_counter == 3 or p_counter == 1:
            #print(combo)
            print(c)
            symplectic_effect(c,phases=False)
            for ini_s in ini_ss:
                print(ini_s)
            print()
            print()
            print()

        if len(options) == 0:
            break

for i,l in enumerate(gate_options):
    print('Option ',i,': ',modes[i])
    l_sorted = sorted(l,key=lambda c: len(c.gates))
    print(l_sorted[0])
    print()
    for j in l_sorted[1:]:
        print(j)
        symplectic_effect(j,phases=False)
        print()
    print()
    print('----------------------------------')

Option  0 :  SSSS
H [1]
S [1]


S [0]
H [1]
S [1]
S [0]



(X**r1*Z**s1)x(X**s2*Z**(r2 + s2))


H [1]
SUM [1, 0]
S [1]
SUM [1, 0]



(X**r1*Z**s1)x(X**s2*Z**(r2 + s2))


SUM [0, 1]
SUM [1, 0]
S [0]
SUM [1, 0]
SUM [0, 1]



...


SUM [0, 1]
SUM [1, 0]
S [0]
SUM [1, 0]
SUM [0, 1]
SUM [1, 0]
S [0]
SUM [1, 0]
H [1]
SUM [0, 1]
S [0]
SUM [0, 1]
S [1]
H [1]
S [1]
SUM [0, 1]



(X**r1*Z**s1)x(X**(r2 + s2)*Z**r2)


SUM [0, 1]
H [1]
S [1]
H [1]
SUM [0, 1]
SUM [1, 0]
S [0]
SUM [1, 0]
H [1]
SUM [0, 1]
S [0]
SUM [0, 1]
S [1]
H [1]
S [1]
SUM [0, 1]



(X**r1*Z**s1)x(X**s2*Z**(r2 + s2))



----------------------------------


In [8]:
def add_phase(P,cq,qubit,phase_mode):
    if phase_mode == 'SSSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())])
        return(C)
    elif phase_mode == 'DDSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(cq,qubit,2),S(qubit,2),H(qubit,2),S(qubit,2),CX(cq,qubit,2)])
        return(C)
    elif phase_mode == 'DSDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(cq,qubit,2),S(qubit,2),CX(cq,qubit,2),H(qubit,2),CX(cq,qubit,2),S(cq,2)])
        return(C)
    elif phase_mode == 'DSSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[S(qubit,2),CX(cq,qubit,2),S(qubit,2),H(qubit,2),S(qubit,2),CX(cq,qubit,2)])
        return(C)
    elif phase_mode == 'SDDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[S(qubit,2),H(qubit,2),CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit,2),S(cq,2)])
        return(C)
    elif phase_mode == 'SDSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit,2),S(cq,2)])
        return(C)    
    elif phase_mode == 'SSDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[H(qubit,2),CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit,2),S(cq,2)])
        return(C) 
    elif phase_mode == 'DDDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(qubit,cq,2),S(cq,2),CX(qubit,cq,2),H(qubit,2),CX(cq,qubit,2),S(cq,2),CX(cq,qubit,2),S(qubit,2),H(qubit,2),S(qubit,2),CX(cq,qubit,2)])
        return(C)     

In [10]:
### add_r2

list_addr2 = []

# I: S
# X: D
# Z: S
# Y: D

'''
I  ->  Z
X  ->  X
Z  ->  I
Y  ->  Y
'''
list_addr2.append(Circuit(dimensions=[2,2], gates=[CX(1,0,2),H(0,2),CX(1,0,2)])) # checked

'''
I  ->  X
X  ->  Z
Z  ->  I
Y  ->  Y
'''
list_addr2.append(Circuit(dimensions=[2,2], gates=[CX(1,0,2),H(1,2),CX(0,1,2)])) #checked
list_addr2.append(Circuit(dimensions=[2,2], gates=[H(0,2),CX(1,0,2),H(1,2),CX(0,1,2)])) #checked
list_addr2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),CX(1,0,2),H(0,2),CX(0,1,2),CX(1,0,2)])) #checked
list_addr2.append(Circuit(dimensions=[2,2], gates=[CX(1,0,2),S(0,2),H(1,2),CX(0,1,2),S(0,2)])) # checked

'''
I  ->  Z
X  ->  X
Z  ->  I
Y  ->  Y
'''
list_addr2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),CX(1,0,2),H(1,2),CX(1,0,2),CX(0,1,2)])) #checked 


'''
I  ->  X
X  ->  Y
Z  ->  I
Y  ->  Z
'''
list_addr2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),CX(1,0,2),S(0,2),H(0,2),CX(0,1,2),CX(1,0,2)])) # checked

'''
I  ->  Z
X  ->  Y
Z  ->  I
Y  ->  X
'''
list_addr2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),CX(1,0,2),S(0,2),H(1,2),CX(1,0,2),CX(0,1,2)])) # checked

'''
I  ->  Y
X  ->  X
Z  ->  I
Y  ->  Z
'''
list_addr2.append(Circuit(dimensions=[2,2], gates=[CX(1,0,2),S(0,2),H(1,2),CX(0,1,2),S(1,2),H(1,2),S(0,2)])) # checked


# I: S
# X: S
# Z: S
# Y: S

'''
I  ->  Z
X  ->  I
Z  ->  Y
Y  ->  X
'''
list_addr2.append(Circuit(dimensions=[2,2], gates=[S(0,2),CX(1,0,2),S(0,2)])) # checked

'''
I  ->  Y
X  ->  X
Z  ->  I
Y  ->  Z
'''
list_addr2.append(Circuit(dimensions=[2,2], gates=[S(0,2),CX(1,0,2),S(0,2),H(1,2),S(1,2)])) # checked

'''
I  ->  Z
X  ->  X
Z  ->  I
Y  ->  Y
'''
list_addr2.append(Circuit(dimensions=[2,2], gates=[S(0,2),CX(1,0,2),S(0,2),S(1,2)])) # checked

'''
I  ->  Z
X  ->  Y
Z  ->  I
Y  ->  X
'''
list_addr2.append(Circuit(dimensions=[2,2], gates=[S(0,2),CX(1,0,2),S(0,2),H(0,2)])) # checked
list_addr2.append(Circuit(dimensions=[2,2], gates=[S(0,2),CX(1,0,2),CX(0,1,2),S(0,2),CX(0,1,2)])) # checked

'''
I  ->  X
X  ->  Y
Z  ->  I
Y  ->  Z
'''
list_addr2.append(Circuit(dimensions=[2,2], gates=[S(0,2),CX(1,0,2),S(0,2),H(1,2)])) # checked
list_addr2.append(Circuit(dimensions=[2,2], gates=[S(0,2),CX(1,0,2),H(1,2),S(0,2)])) # checked
list_addr2.append(Circuit(dimensions=[2,2], gates=[S(0,2),CX(1,0,2),S(0,2),H(1,2),H(0,2)])) # checked

# I: D
# X: D
# Z: D
# Y: D
'''
I  ->  Z
X  ->  Y
Z  ->  I
Y  ->  X
'''
list_addr2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),S(0,2),S(1,2),CX(0,1,2),CX(1,0,2)])) # checked
list_addr2.append(Circuit(dimensions=[2,2], gates=[CX(1,0,2),CX(0,1,2),S(0,2),S(1,2),CX(0,1,2)])) # checked
list_addr2.append(Circuit(dimensions=[2,2], gates=[CX(1,0,2),CX(0,1,2),S(1,2),CX(0,1,2),S(0,2)])) # checked

'''
I  ->  X
X  ->  Y
Z  ->  I
Y  ->  Z
'''
list_addr2.append(Circuit(dimensions=[2,2], gates=[CX(1,0,2),CX(0,1,2),S(0,2),S(1,2),H(0,2),H(1,2),CX(1,0,2)])) # checked
list_addr2 = sorted(list_addr2,key=lambda c: len(c.gates))







In [11]:
start_paulis = [PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[0,0],standardise=False),
               PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[0,0],standardise=False),
               PauliSum(['x1z0 x0z1', 'x0z1 x0z1'], dimensions=[2, 2], phases=[0,0],standardise=False),
               PauliSum(['x1z0 x1z1', 'x0z1 x1z1'], dimensions=[2, 2], phases=[0,0],standardise=False)]

modes = ['SSSS','DDSS','DSDS','DSSD','SDDS','SDSD','SSDD','DDDD'] 
options = [0,1,2,3,4,5,6,7]
gate_options = [[],[],[],[],[],[],[],[]]

for mode in modes:
    for i,ci in enumerate(list_addr2):
        c = Circuit(dimensions=[2,2])
        id_c = add_phase(start_paulis[0],0,1,mode)
        for gj in ci.gates:
            c.add_gate(gj)
        for gj in id_c.gates:
            c.add_gate(gj)
        #print(c)
        #symplectic_effect(c,phases=False)
        '''
        for ip,p in enumerate(start_paulis):
            P_temp = c.act(p)
            if ip == 0:
                ini_s = 'I'
            elif ip == 1:
                ini_s = 'X'
            elif ip == 2:
                ini_s = 'Z'
            elif ip == 3:
                ini_s = 'Y'

            if P_temp.x_exp[0,1] == 0:
                if P_temp.z_exp[0,1] == 0:
                    print(ini_s,' -> ','I')
                else:
                    print(ini_s,' -> ','Z')
            elif P_temp.x_exp[0,1] == 1:
                if P_temp.z_exp[0,1] == 0:
                    print(ini_s,' -> ','X')
                else:
                    print(ini_s,' -> ','Y')

            phases = P_temp.phases
            if phases[0] == phases[1]:
                print('Same Phase')
            else:
                print('Different Phase')
        print()
        print()
        print()
        '''

        p_counter = 0
        p_ss = ''
        ini_ss = []
        for ip,p in enumerate(start_paulis):
            P_temp = c.act(p)
            if ip == 0:
                ini_s = 'I'
            elif ip == 1:
                ini_s = 'X'
            elif ip == 2:
                ini_s = 'Z'
            elif ip == 3:
                ini_s = 'Y'

            if P_temp.x_exp[0,1] == 0:
                if P_temp.z_exp[0,1] == 0:
                    ini_s += '-> I'
                    #print(ini_s,' -> ','I')
                    pass
                else:
                    ini_s += '-> Z'
                    #print(ini_s,' -> ','Z')
                    pass
            elif P_temp.x_exp[0,1] == 1:
                if P_temp.z_exp[0,1] == 0:
                    ini_s += '-> X'
                    #print(ini_s,' -> ','X')
                    pass
                else:
                    ini_s += '-> Y'
                    #print(ini_s,' -> ','Y')
                    pass
            ini_ss.append(ini_s)
            phases = P_temp.phases
            if phases[0] == phases[1]:
                #print('Same Phase')
                p_ss += 'S'
                p_counter += 1
            else:
                p_ss += 'D'
                #print('Different Phase')
                pass
        if p_ss == 'SSSS' and 0 in options:
            #print('Option 0: SSSS')
            #print(c)
            #print()
            #options.remove(0)
            gate_options[0].append(c)
        elif p_ss == 'DDSS' and 1 in options:
            #print('Option 1: DDSS')
            #print(c)
            #print()
            #options.remove(1)
            gate_options[1].append(c)
        elif p_ss == 'DSDS' and 2 in options:
            #print('Option 2: DSDS')
            #print(c)
            #print()
            #options.remove(2)
            gate_options[2].append(c)
        elif p_ss == 'DSSD' and 3 in options:
            #print('Option 3: DSSD')
            #print(c)
            #print()
            #options.remove(3)
            gate_options[3].append(c)
        elif p_ss == 'SDDS' and 4 in options:
            #print('Option 4: SDDS')
            #print(c)
            #print()
            #options.remove(4)
            gate_options[4].append(c)
        elif p_ss == 'SDSD' and 5 in options:
            #print('Option 5: SDSD')
            #print(c)
            #print()
            #options.remove(5)
            gate_options[5].append(c)
        elif p_ss == 'SSDD' and 6 in options:
            #print('Option 6: SSDD')
            #print(c)
            #print()
            #options.remove(6)
            gate_options[6].append(c)
        elif p_ss == 'DDDD' and 7 in options:
            #print('Option 7: DDDD')
            #print(c)
            #print()
            #options.remove(7)
            gate_options[7].append(c)

        if p_counter == 3 or p_counter == 1:
            #print(combo)
            print(c)
            symplectic_effect(c,phases=False)
            for ini_s in ini_ss:
                print(ini_s)
            print()
            print()
            print()

        if len(options) == 0:
            break

for i,l in enumerate(gate_options):
    print('Option ',i,': ',modes[i])
    l_sorted = sorted(l,key=lambda c: len(c.gates))
    print(l_sorted[0])
    print()

Option  0 :  SSSS
S [0]
SUM [1, 0]
S [0]


Option  1 :  DDSS
SUM [1, 0]
H [1]
SUM [0, 1]
SUM [1, 0]
S [0]
SUM [1, 0]
H [1]
SUM [0, 1]
S [0]


Option  2 :  DSDS
S [0]
SUM [1, 0]
S [0]
H [1]
SUM [0, 1]
S [1]
H [1]
S [1]
SUM [0, 1]


Option  3 :  DSSD
SUM [0, 1]
S [0]
S [1]
SUM [0, 1]
SUM [1, 0]
S [1]
SUM [0, 1]
S [1]
H [1]
S [1]
SUM [0, 1]


Option  4 :  SDDS
S [0]
SUM [1, 0]
S [0]
S [1]
SUM [0, 1]
S [1]
H [1]
S [1]
SUM [0, 1]


Option  5 :  SDSD
SUM [1, 0]
H [0]
SUM [1, 0]


Option  6 :  SSDD
SUM [1, 0]
H [0]
SUM [1, 0]
SUM [0, 1]
S [1]
H [1]
S [1]
SUM [0, 1]


Option  7 :  DDDD
SUM [0, 1]
S [0]
S [1]
SUM [0, 1]
SUM [1, 0]




In [ ]:
def add_r2(P,q0,q1,phase_mode):
    if phase_mode == 'SSSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())], 
                    gates=[S(q0,2),CX(q1,q0,2),S(q0,2)])
        return(C)
    elif phase_mode == 'DDSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q1,q0,2),H(q1,2),CX(q0,q1,2),CX(q1,q0,2),S(q0,2),CX(q1,q0,2),H(q1,2),CX(q0,q1,2),S(q0,2)])
        return(C)
    elif phase_mode == 'DSDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[S(q0,2),CX(q1,q0,2),S(q0,2),H(q1,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'DSSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),S(q0,2),S(q1,2),CX(q0,q1,2),CX(q1,q0,2),S(q1,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'SDDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[S(q0,2),CX(q1,q0,2),S(q0,2),S(q1,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'SDSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q1,q0,2),H(q0,2),CX(q1,q0,2)])
        return(C)    
    elif phase_mode == 'SSDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q1,q0,2),H(q0,2),CX(q1,q0,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C) 
    elif phase_mode == 'DDDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),S(q0,2),S(q1,2),CX(q0,q1,2),CX(q1,q0,2)])
        return(C)   

for i,mode in enumerate(modes):
    c = add_r2(start_paulis[0],0,1,mode)
    print(c)
    symplectic_effect(c,phases=False)
    for ip,p in enumerate(start_paulis):
        P_temp = c.act(p)
        if ip == 0:
            ini_s = 'I'
        elif ip == 1:
            ini_s = 'X'
        elif ip == 2:
            ini_s = 'Z'
        elif ip == 3:
            ini_s = 'Y'

        if P_temp.x_exp[0,1] == 0:
            if P_temp.z_exp[0,1] == 0:
                print(ini_s,' -> ','I')
            else:
                print(ini_s,' -> ','Z')
        elif P_temp.x_exp[0,1] == 1:
            if P_temp.z_exp[0,1] == 0:
                print(ini_s,' -> ','X')
            else:
                print(ini_s,' -> ','Y')

        phases = P_temp.phases
        if phases[0] == phases[1]:
            print('Same Phase')
        else:
            print('Different Phase')
    print()
    print()
    print() 

S [0]
SUM [1, 0]
S [0]



(X**(r1 + r2)*Z**(r2 + s1))x(X**r2*Z**(r1 + s1 + s2))

I  ->  Z
Same Phase
X  ->  Y
Same Phase
Z  ->  I
Same Phase
Y  ->  X
Same Phase



SUM [1, 0]
H [1]
SUM [0, 1]
SUM [1, 0]
S [0]
SUM [1, 0]
H [1]
SUM [0, 1]
S [0]



(X**(r1 + r2)*Z**(r2 + s1))x(X**(r1 + s1 + s2)*Z**(r1 + r2 + s1 + s2))

I  ->  Y
Different Phase
X  ->  X
Different Phase
Z  ->  I
Same Phase
Y  ->  Z
Same Phase



S [0]
SUM [1, 0]
S [0]
H [1]
SUM [0, 1]
S [1]
H [1]
S [1]
SUM [0, 1]



...

I  ->  Z
Same Phase
X  ->  X
Different Phase
Z  ->  I
Same Phase
Y  ->  Y
Different Phase



SUM [1, 0]
H [0]
SUM [1, 0]
SUM [0, 1]
S [1]
H [1]
S [1]
SUM [0, 1]



(X**(r2 + s1)*Z**(r1 + r2))x(X**(r1 + s1 + s2)*Z**(r1 + r2 + s1 + s2))

I  ->  Y
Same Phase
X  ->  X
Same Phase
Z  ->  I
Different Phase
Y  ->  Z
Different Phase



SUM [0, 1]
S [0]
S [1]
SUM [0, 1]
SUM [1, 0]



(X**(r1 + r2)*Z**(r2 + s1))x(X**r2*Z**(r1 + s1 + s2))

I  ->  Z
Different Phase
X  ->  Y
Different Phase
Z  ->  I
Different Phase
Y  ->  X
Different Phase





In [8]:
### add_s2

list_adds2 = []

# I: S
# X: S
# Z: D
# Y: D

'''
I  ->  X
X  ->  I
Z  ->  Z
Y  ->  Y
'''
list_adds2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),H(0,2),CX(0,1,2)])) # checked
list_adds2.append(Circuit(dimensions=[2,2], gates=[H(0,2),CX(0,1,2),H(0,2),CX(0,1,2)])) # checked
list_adds2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),H(0,2),S(0,2),CX(0,1,2),S(0,2)])) # checked

'''
I  ->  Z
X  ->  I
Z  ->  X
Y  ->  Y
'''
list_adds2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),H(1,2),CX(1,0,2)])) # checked
list_adds2.append(Circuit(dimensions=[2,2], gates=[H(0,2),CX(0,1,2),H(1,2),CX(1,0,2)])) # checked
list_adds2.append(Circuit(dimensions=[2,2], gates=[CX(1,0,2),CX(0,1,2),H(0,2),CX(1,0,2),CX(0,1,2)])) # checked
list_adds2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),H(0,2),S(0,2),CX(0,1,2),H(1,2),S(0,2)])) # checked

'''
I  ->  Z
X  ->  I
Z  ->  Y
Y  ->  X
'''
list_adds2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),H(1,2),S(1,2),CX(1,0,2)])) # checked
list_adds2.append(Circuit(dimensions=[2,2], gates=[H(0,2),CX(0,1,2),H(1,2),S(1,2),CX(1,0,2)])) # checked


'''
I  ->  X
X  ->  I
Z  ->  Y
Y  ->  Z
'''

list_adds2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),H(0,2),H(1,2),S(1,2),H(1,2),CX(0,1,2)])) # checked
list_adds2.append(Circuit(dimensions=[2,2], gates=[H(1,2),CX(1,0,2),S(1,2),H(1,2),CX(0,1,2)])) # checked


# I: D
# X: D
# Z: S
# Y: S

'''
I  ->  Z
X  ->  I
Z  ->  X
Y  ->  Y
'''
list_adds2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),CX(1,0,2),H(0,2),CX(0,1,2),H(1,2),CX(0,1,2)])) # checked

# I: S
# X: D
# Z: S
# Y: D

'''
I  ->  Y
X  ->  I
Z  ->  Z
Y  ->  X
'''
list_adds2.append(Circuit(dimensions=[2,2], gates=[CX(1,0,2),S(0,2),H(1,2),CX(0,1,2),CX(1,0,2),S(0,2)]))
list_adds2 = sorted(list_adds2,key=lambda c: len(c.gates))


In [9]:
start_paulis = [PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[0,0],standardise=False),
               PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[0,0],standardise=False),
               PauliSum(['x1z0 x0z1', 'x0z1 x0z1'], dimensions=[2, 2], phases=[0,0],standardise=False),
               PauliSum(['x1z0 x1z1', 'x0z1 x1z1'], dimensions=[2, 2], phases=[0,0],standardise=False)]

modes = ['SSSS','DDSS','DSDS','DSSD','SDDS','SDSD','SSDD','DDDD'] 
options = [0,1,2,3,4,5,6,7]
gate_options = [[],[],[],[],[],[],[],[]]

for mode in modes:
    for i,ci in enumerate(list_adds2):
        c = Circuit(dimensions=[2,2])
        id_c = add_phase(start_paulis[0],0,1,mode)
        for gj in ci.gates:
            c.add_gate(gj)
        for gj in id_c.gates:
            c.add_gate(gj)
        #print(c)
        #symplectic_effect(c,phases=False)
        '''
        for ip,p in enumerate(start_paulis):
            P_temp = c.act(p)
            if ip == 0:
                ini_s = 'I'
            elif ip == 1:
                ini_s = 'X'
            elif ip == 2:
                ini_s = 'Z'
            elif ip == 3:
                ini_s = 'Y'

            if P_temp.x_exp[0,1] == 0:
                if P_temp.z_exp[0,1] == 0:
                    print(ini_s,' -> ','I')
                else:
                    print(ini_s,' -> ','Z')
            elif P_temp.x_exp[0,1] == 1:
                if P_temp.z_exp[0,1] == 0:
                    print(ini_s,' -> ','X')
                else:
                    print(ini_s,' -> ','Y')

            phases = P_temp.phases
            if phases[0] == phases[1]:
                print('Same Phase')
            else:
                print('Different Phase')
        print()
        print()
        print()
        '''

        p_counter = 0
        p_ss = ''
        ini_ss = []
        for ip,p in enumerate(start_paulis):
            P_temp = c.act(p)
            if ip == 0:
                ini_s = 'I'
            elif ip == 1:
                ini_s = 'X'
            elif ip == 2:
                ini_s = 'Z'
            elif ip == 3:
                ini_s = 'Y'

            if P_temp.x_exp[0,1] == 0:
                if P_temp.z_exp[0,1] == 0:
                    ini_s += '-> I'
                    #print(ini_s,' -> ','I')
                    pass
                else:
                    ini_s += '-> Z'
                    #print(ini_s,' -> ','Z')
                    pass
            elif P_temp.x_exp[0,1] == 1:
                if P_temp.z_exp[0,1] == 0:
                    ini_s += '-> X'
                    #print(ini_s,' -> ','X')
                    pass
                else:
                    ini_s += '-> Y'
                    #print(ini_s,' -> ','Y')
                    pass
            ini_ss.append(ini_s)
            phases = P_temp.phases
            if phases[0] == phases[1]:
                #print('Same Phase')
                p_ss += 'S'
                p_counter += 1
            else:
                p_ss += 'D'
                #print('Different Phase')
                pass
        if p_ss == 'SSSS' and 0 in options:
            #print('Option 0: SSSS')
            #print(c)
            #print()
            #options.remove(0)
            gate_options[0].append(c)
        elif p_ss == 'DDSS' and 1 in options:
            #print('Option 1: DDSS')
            #print(c)
            #print()
            #options.remove(1)
            gate_options[1].append(c)
        elif p_ss == 'DSDS' and 2 in options:
            #print('Option 2: DSDS')
            #print(c)
            #print()
            #options.remove(2)
            gate_options[2].append(c)
        elif p_ss == 'DSSD' and 3 in options:
            #print('Option 3: DSSD')
            #print(c)
            #print()
            #options.remove(3)
            gate_options[3].append(c)
        elif p_ss == 'SDDS' and 4 in options:
            #print('Option 4: SDDS')
            #print(c)
            #print()
            #options.remove(4)
            gate_options[4].append(c)
        elif p_ss == 'SDSD' and 5 in options:
            #print('Option 5: SDSD')
            #print(c)
            #print()
            #options.remove(5)
            gate_options[5].append(c)
        elif p_ss == 'SSDD' and 6 in options:
            #print('Option 6: SSDD')
            #print(c)
            #print()
            #options.remove(6)
            gate_options[6].append(c)
        elif p_ss == 'DDDD' and 7 in options:
            #print('Option 7: DDDD')
            #print(c)
            #print()
            #options.remove(7)
            gate_options[7].append(c)

        if p_counter == 3 or p_counter == 1:
            #print(combo)
            print(c)
            symplectic_effect(c,phases=False)
            for ini_s in ini_ss:
                print(ini_s)
            print()
            print()
            print()

        if len(options) == 0:
            break

for i,l in enumerate(gate_options):
    print('Option ',i,': ',modes[i])
    l_sorted = sorted(l,key=lambda c: len(c.gates))
    print(l_sorted[0])
    print()

Option  0 :  SSSS
SUM [0, 1]
H [1]
SUM [1, 0]
SUM [1, 0]
S [0]
SUM [1, 0]
H [1]
SUM [0, 1]
S [0]


Option  1 :  DDSS
SUM [0, 1]
SUM [1, 0]
H [0]
SUM [0, 1]
H [1]
SUM [0, 1]


Option  2 :  DSDS
SUM [0, 1]
H [0]
SUM [0, 1]
SUM [1, 0]
S [0]
SUM [1, 0]
H [1]
SUM [0, 1]
S [0]


Option  3 :  DSSD
SUM [0, 1]
H [0]
SUM [0, 1]
S [1]
H [1]
SUM [1, 0]
S [0]
SUM [1, 0]
H [1]
SUM [0, 1]
S [0]


Option  4 :  SDDS
SUM [0, 1]
H [1]
S [1]
SUM [1, 0]
SUM [0, 1]
S [1]
H [1]
S [1]
SUM [0, 1]


Option  5 :  SDSD
SUM [1, 0]
S [0]
H [1]
SUM [0, 1]
SUM [1, 0]
S [0]


Option  6 :  SSDD
SUM [0, 1]
H [0]
SUM [0, 1]


Option  7 :  DDDD
SUM [0, 1]
H [0]
SUM [0, 1]
SUM [0, 1]
S [1]
H [1]
S [1]
SUM [0, 1]




In [10]:
def add_s2(P,q0,q1,phase_mode):
    if phase_mode == 'SSSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())], 
                    gates=[CX(q0,q1,2),H(q1,2),S(q0,2),CX(q1,q0,2),H(q1,2),CX(q0,q1,2),S(q0,2)])
        return(C)
    elif phase_mode == 'DDSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),CX(q1,q0,2),H(q0,2),CX(q0,q1,2),H(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'DSDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),H(q0,2),CX(q0,q1,2),CX(q1,q0,2),S(q0,2),CX(q1,q0,2),H(q1,2),CX(q0,q1,2),S(q0,2)])
        return(C)
    elif phase_mode == 'DSSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),H(q0,2),CX(q0,q1,2),S(q1,2),H(q1,2),CX(q1,q0,2),S(q0,2),CX(q1,q0,2),H(q1,2),CX(q0,q1,2),S(q0,2)])
        return(C)
    elif phase_mode == 'SDDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),H(q1,2),S(q1,2),CX(q1,q0,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'SDSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q1,q0,2),S(q0,2),H(q1,2),CX(q0,q1,2),CX(q1,q0,2),S(q0,2)])
        return(C)    
    elif phase_mode == 'SSDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),H(q0,2),CX(q0,q1,2)])
        return(C) 
    elif phase_mode == 'DDDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),H(q0,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)  
    

for i,mode in enumerate(modes):
    c = add_s2(start_paulis[0],0,1,mode)
    print(c)
    symplectic_effect(c,phases=False)
    for ip,p in enumerate(start_paulis):
        P_temp = c.act(p)
        if ip == 0:
            ini_s = 'I'
        elif ip == 1:
            ini_s = 'X'
        elif ip == 2:
            ini_s = 'Z'
        elif ip == 3:
            ini_s = 'Y'

        if P_temp.x_exp[0,1] == 0:
            if P_temp.z_exp[0,1] == 0:
                print(ini_s,' -> ','I')
            else:
                print(ini_s,' -> ','Z')
        elif P_temp.x_exp[0,1] == 1:
            if P_temp.z_exp[0,1] == 0:
                print(ini_s,' -> ','X')
            else:
                print(ini_s,' -> ','Y')

        phases = P_temp.phases
        if phases[0] == phases[1]:
            print('Same Phase')
        else:
            print('Different Phase')
    print()
    print()
    print()

SUM [0, 1]
H [1]
S [0]
SUM [1, 0]
H [1]
SUM [0, 1]
S [0]



(X**(r1 + s2)*Z**(s1 + s2))x(X**(r1 + r2 + s1)*Z**s2)

I  ->  X
Same Phase
X  ->  I
Same Phase
Z  ->  Y
Same Phase
Y  ->  Z
Same Phase



SUM [0, 1]
SUM [1, 0]
H [0]
SUM [0, 1]
H [1]
SUM [0, 1]



(X**(s1 + s2)*Z**(r1 + s2))x(X**s2*Z**(r1 + r2 + s1 + s2))

I  ->  Z
Different Phase
X  ->  I
Different Phase
Z  ->  X
Same Phase
Y  ->  Y
Same Phase



SUM [0, 1]
H [0]
SUM [0, 1]
SUM [1, 0]
S [0]
SUM [1, 0]
H [1]
SUM [0, 1]
S [0]



...

I  ->  Y
Same Phase
X  ->  I
Different Phase
Z  ->  Z
Same Phase
Y  ->  X
Different Phase



SUM [0, 1]
H [0]
SUM [0, 1]



(X**(s1 + s2)*Z**(r1 + s2))x(X**(r1 + r2 + s1 + s2)*Z**s2)

I  ->  X
Same Phase
X  ->  I
Same Phase
Z  ->  Z
Different Phase
Y  ->  Y
Different Phase



SUM [0, 1]
H [0]
S [1]
H [1]
S [1]
SUM [0, 1]



(X**(s1 + s2)*Z**(r1 + s2))x(X**(r1 + r2 + s1)*Z**s2)

I  ->  X
Different Phase
X  ->  I
Different Phase
Z  ->  Y
Different Phase
Y  ->  Z
Different Phase





In [10]:
# add_r2s2

list_addr2s2 = []

# I: S
# X: S
# Z: S
# Y: S

'''
I  ->  Y
X  ->  X
Z  ->  Z
Y  ->  I
'''
list_addr2s2.append(Circuit(dimensions=[2,2], gates=[S(0,2),CX(1,0,2),H(1,2),CX(1,0,2),S(0,2)]))
list_addr2s2.append(Circuit(dimensions=[2,2], gates=[S(0,2),CX(1,0,2),H(1,2),CX(1,0,2),S(0,2),H(0,2)]))

'''
I  ->  Y
X  ->  Z
Z  ->  X
Y  ->  I
'''
list_addr2s2.append(Circuit(dimensions=[2,2], gates=[S(0,2),CX(1,0,2),H(1,2),CX(1,0,2),S(0,2),H(1,2)]))

'''
I  ->  X
X  ->  Y
Z  ->  Z
Y  ->  I
'''
list_addr2s2.append(Circuit(dimensions=[2,2], gates=[S(0,2),CX(1,0,2),H(1,2),CX(1,0,2),S(0,2),S(1,2)]))

'''
I  ->  Z
X  ->  Y
Z  ->  X
Y  ->  I
'''
list_addr2s2.append(Circuit(dimensions=[2,2], gates=[S(0,2),H(1,2),S(1,2),H(1,2),CX(1,0,2),S(0,2)]))


# I: S
# X: D
# Z: D
# Y: S

'''
I  ->  X
X  ->  Y
Z  ->  Z
Y  ->  I
'''
list_addr2s2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),CX(1,0,2),S(0,2),CX(1,0,2),H(0,2),CX(0,1,2)]))

'''
I  ->  Z
X  ->  Y
Z  ->  X
Y  ->  I
'''
list_addr2s2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),CX(1,0,2),S(0,2),CX(1,0,2),H(1,2),CX(1,0,2)]))

# I: S
# X: S
# Z: D
# Y: D

'''
I  ->  X
X  ->  Y
Z  ->  Z
Y  ->  I
'''
list_addr2s2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),S(0,2),S(1,2),H(0,2),CX(1,0,2),CX(0,1,2)]))
list_addr2s2.append(Circuit(dimensions=[2,2], gates=[CX(1,0,2),CX(0,1,2),S(0,2),S(1,2),H(1,2),CX(1,0,2)]))


'''
I  ->  Z
X  ->  Y
Z  ->  X
Y  ->  I
'''
list_addr2s2.append(Circuit(dimensions=[2,2], gates=[CX(1,0,2),CX(0,1,2),S(0,2),S(1,2),H(0,2),CX(0,1,2)]))



# I: D
# X: D
# Z: S
# Y: S
'''
I  ->  Z
X  ->  Y
Z  ->  X
Y  ->  I
'''
list_addr2s2.append(Circuit(dimensions=[2,2], gates=[S(0,2),CX(1,0,2),CX(0,1,2),S(0,2),H(0,2),CX(0,1,2)]))

'''
I  ->  X
X  ->  Y
Z  ->  Z
Y  ->  I
'''
list_addr2s2.append(Circuit(dimensions=[2,2], gates=[S(0,2),CX(1,0,2),CX(0,1,2),S(0,2),H(1,2),CX(1,0,2)]))

# I: S
# X: D
# Z: S
# Y: D

'''
I  ->  Z
X  ->  X
Z  ->  Y
Y  ->  I
'''
list_addr2s2.append(Circuit(dimensions=[2,2], gates=[CX(1,0,2),CX(0,1,2),S(0,2),CX(1,0,2),H(0,2),CX(0,1,2),S(0,2)]))


# I: D
# X: D
# Z: D
# Y: D

'''
I  ->  Z
X  ->  X
Z  ->  Y
Y  ->  I
'''
list_addr2s2.append(Circuit(dimensions=[2,2], gates=[CX(0,1,2),CX(1,0,2),S(0,2),H(0,2),H(1,2),S(1,2),CX(0,1,2),S(0,2)]))
list_addr2s2 = sorted(list_addr2s2,key=lambda c: len(c.gates))

In [11]:
start_paulis = [PauliSum(['x1z0 x0z0', 'x0z1 x0z0'], dimensions=[2, 2], phases=[0,0],standardise=False),
               PauliSum(['x1z0 x1z0', 'x0z1 x1z0'], dimensions=[2, 2], phases=[0,0],standardise=False),
               PauliSum(['x1z0 x0z1', 'x0z1 x0z1'], dimensions=[2, 2], phases=[0,0],standardise=False),
               PauliSum(['x1z0 x1z1', 'x0z1 x1z1'], dimensions=[2, 2], phases=[0,0],standardise=False)]

modes = ['SSSS','DDSS','DSDS','DSSD','SDDS','SDSD','SSDD','DDDD'] 
options = [0,1,2,3,4,5,6,7]
gate_options = [[],[],[],[],[],[],[],[]]

for mode in modes:
    for i,ci in enumerate(list_addr2s2):
        c = Circuit(dimensions=[2,2])
        id_c = add_phase(start_paulis[0],0,1,mode)
        for gj in ci.gates:
            c.add_gate(gj)
        for gj in id_c.gates:
            c.add_gate(gj)
        #print(c)
        #symplectic_effect(c,phases=False)
        '''
        for ip,p in enumerate(start_paulis):
            P_temp = c.act(p)
            if ip == 0:
                ini_s = 'I'
            elif ip == 1:
                ini_s = 'X'
            elif ip == 2:
                ini_s = 'Z'
            elif ip == 3:
                ini_s = 'Y'

            if P_temp.x_exp[0,1] == 0:
                if P_temp.z_exp[0,1] == 0:
                    print(ini_s,' -> ','I')
                else:
                    print(ini_s,' -> ','Z')
            elif P_temp.x_exp[0,1] == 1:
                if P_temp.z_exp[0,1] == 0:
                    print(ini_s,' -> ','X')
                else:
                    print(ini_s,' -> ','Y')

            phases = P_temp.phases
            if phases[0] == phases[1]:
                print('Same Phase')
            else:
                print('Different Phase')
        print()
        print()
        print()
        '''

        p_counter = 0
        p_ss = ''
        ini_ss = []
        for ip,p in enumerate(start_paulis):
            P_temp = c.act(p)
            if ip == 0:
                ini_s = 'I'
            elif ip == 1:
                ini_s = 'X'
            elif ip == 2:
                ini_s = 'Z'
            elif ip == 3:
                ini_s = 'Y'

            if P_temp.x_exp[0,1] == 0:
                if P_temp.z_exp[0,1] == 0:
                    ini_s += '-> I'
                    #print(ini_s,' -> ','I')
                    pass
                else:
                    ini_s += '-> Z'
                    #print(ini_s,' -> ','Z')
                    pass
            elif P_temp.x_exp[0,1] == 1:
                if P_temp.z_exp[0,1] == 0:
                    ini_s += '-> X'
                    #print(ini_s,' -> ','X')
                    pass
                else:
                    ini_s += '-> Y'
                    #print(ini_s,' -> ','Y')
                    pass
            ini_ss.append(ini_s)
            phases = P_temp.phases
            if phases[0] == phases[1]:
                #print('Same Phase')
                p_ss += 'S'
                p_counter += 1
            else:
                p_ss += 'D'
                #print('Different Phase')
                pass
        if p_ss == 'SSSS' and 0 in options:
            #print('Option 0: SSSS')
            #print(c)
            #print()
            #options.remove(0)
            gate_options[0].append(c)
        elif p_ss == 'DDSS' and 1 in options:
            #print('Option 1: DDSS')
            #print(c)
            #print()
            #options.remove(1)
            gate_options[1].append(c)
        elif p_ss == 'DSDS' and 2 in options:
            #print('Option 2: DSDS')
            #print(c)
            #print()
            #options.remove(2)
            gate_options[2].append(c)
        elif p_ss == 'DSSD' and 3 in options:
            #print('Option 3: DSSD')
            #print(c)
            #print()
            #options.remove(3)
            gate_options[3].append(c)
        elif p_ss == 'SDDS' and 4 in options:
            #print('Option 4: SDDS')
            #print(c)
            #print()
            #options.remove(4)
            gate_options[4].append(c)
        elif p_ss == 'SDSD' and 5 in options:
            #print('Option 5: SDSD')
            #print(c)
            #print()
            #options.remove(5)
            gate_options[5].append(c)
        elif p_ss == 'SSDD' and 6 in options:
            #print('Option 6: SSDD')
            #print(c)
            #print()
            #options.remove(6)
            gate_options[6].append(c)
        elif p_ss == 'DDDD' and 7 in options:
            #print('Option 7: DDDD')
            #print(c)
            #print()
            #options.remove(7)
            gate_options[7].append(c)

        if p_counter == 3 or p_counter == 1:
            #print(combo)
            print(c)
            symplectic_effect(c,phases=False)
            for ini_s in ini_ss:
                print(ini_s)
            print()
            print()
            print()

        if len(options) == 0:
            break

for i,l in enumerate(gate_options):
    print('Option ',i,': ',modes[i])
    l_sorted = sorted(l,key=lambda c: len(c.gates))
    print(l_sorted[0])
    print()
    for j in l_sorted[1:]:
        print(j)
        symplectic_effect(j,phases=False)
        print()
    print()
    print('----------------------------------')

Option  0 :  SSSS
S [0]
SUM [1, 0]
H [1]
SUM [1, 0]
S [0]


S [0]
SUM [1, 0]
H [1]
SUM [1, 0]
S [0]
H [0]



(X**(r1 + r2 + s2)*Z**(r2 + s1 + s2))x(X**(r1 + s1 + s2)*Z**(r1 + r2 + s1))


S [0]
SUM [1, 0]
H [1]
SUM [1, 0]
S [0]
H [1]



(X**(r2 + s1 + s2)*Z**(r1 + r2 + s2))x(X**(r1 + r2 + s1)*Z**(r1 + s1 + s2))


S [0]
SUM [1, 0]
H [1]
SUM [1, 0]
S [0]
S [1]



...


S [0]
SUM [1, 0]
H [1]
SUM [1, 0]
S [0]
S [1]
SUM [1, 0]
S [0]
SUM [1, 0]
H [1]
SUM [0, 1]
S [0]
SUM [0, 1]
S [1]
H [1]
S [1]
SUM [0, 1]



(X**(r2 + s1 + s2)*Z**(r1 + r2 + s2))x(X**(r2 + s2)*Z**(r1 + s1 + s2))


S [0]
H [1]
S [1]
H [1]
SUM [1, 0]
S [0]
SUM [1, 0]
S [0]
SUM [1, 0]
H [1]
SUM [0, 1]
S [0]
SUM [0, 1]
S [1]
H [1]
S [1]
SUM [0, 1]



(X**(r1 + r2 + s2)*Z**(r2 + s1 + s2))x(X**(r1 + s1 + s2)*Z**(r2 + s2))



----------------------------------


In [13]:
def add_r2s2(P,q0,q1,phase_mode):
    if phase_mode == 'SSSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())], 
                    gates=[S(q0,2),CX(q1,q0,2),H(q1,2),CX(q1,q0,2),S(q0,2)])
        return(C)
    elif phase_mode == 'DDSS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[S(q0,2),CX(q1,q0,2),CX(q0,q1,2),S(q0,2),H(q0,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'DSDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),S(q0,2),S(q1,2),H(q0,2),CX(q1,q0,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'DSSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[S(q0,2),CX(q1,q0,2),H(q1,2),CX(q1,q0,2),S(q0,2),S(q1,2),CX(q0,q1,2),S(q1,2),H(q1,2),S(q1,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'SDDS':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),CX(q1,q0,2),S(q0,2),CX(q1,q0,2),H(q0,2),CX(q0,q1,2)])
        return(C)
    elif phase_mode == 'SDSD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q1,q0,2),CX(q0,q1,2),S(q0,2),CX(q1,q0,2),H(q0,2),CX(q0,q1,2),S(q0,2)])
        return(C)    
    elif phase_mode == 'SSDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),S(q0,2),S(q1,2),H(q0,2),CX(q1,q0,2),CX(q0,q1,2)])
        return(C) 
    elif phase_mode == 'DDDD':
        C = Circuit(dimensions=[2 for i in range(P.n_qudits())],
                    gates=[CX(q0,q1,2),CX(q1,q0,2),S(q0,2),H(q0,2),H(q1,2),S(q1,2),CX(q0,q1,2),S(q0,2)])
        return(C)  
    

for i,mode in enumerate(modes):
    c = add_r2s2(start_paulis[0],0,1,mode)
    print(c)
    symplectic_effect(c,phases=False)
    for ip,p in enumerate(start_paulis):
        P_temp = c.act(p)
        if ip == 0:
            ini_s = 'I'
        elif ip == 1:
            ini_s = 'X'
        elif ip == 2:
            ini_s = 'Z'
        elif ip == 3:
            ini_s = 'Y'

        if P_temp.x_exp[0,1] == 0:
            if P_temp.z_exp[0,1] == 0:
                print(ini_s,' -> ','I')
            else:
                print(ini_s,' -> ','Z')
        elif P_temp.x_exp[0,1] == 1:
            if P_temp.z_exp[0,1] == 0:
                print(ini_s,' -> ','X')
            else:
                print(ini_s,' -> ','Y')

        phases = P_temp.phases
        if phases[0] == phases[1]:
            print('Same Phase')
        else:
            print('Different Phase')
    print()
    print()
    print()

S [0]
SUM [1, 0]
H [1]
SUM [1, 0]
S [0]



(X**(r2 + s1 + s2)*Z**(r1 + r2 + s2))x(X**(r1 + s1 + s2)*Z**(r1 + r2 + s1))

I  ->  Y
Same Phase
X  ->  X
Same Phase
Z  ->  Z
Same Phase
Y  ->  I
Same Phase



S [0]
SUM [1, 0]
SUM [0, 1]
S [0]
H [0]
SUM [0, 1]



(X**(r1 + r2 + s2)*Z**(r2 + s1 + s2))x(X**(r2 + s2)*Z**(r1 + s1 + s2))

I  ->  Z
Different Phase
X  ->  Y
Different Phase
Z  ->  X
Same Phase
Y  ->  I
Same Phase



SUM [0, 1]
S [0]
S [1]
H [0]
SUM [1, 0]
S [1]
H [1]
S [1]
SUM [0, 1]



...

I  ->  Z
Same Phase
X  ->  X
Different Phase
Z  ->  Y
Same Phase
Y  ->  I
Different Phase



SUM [0, 1]
S [0]
S [1]
H [0]
SUM [1, 0]
SUM [0, 1]



(X**(r2 + s1 + s2)*Z**(r1 + r2 + s2))x(X**(r1 + s1 + s2)*Z**(r2 + s2))

I  ->  X
Same Phase
X  ->  Y
Same Phase
Z  ->  Z
Different Phase
Y  ->  I
Different Phase



SUM [0, 1]
SUM [1, 0]
S [0]
H [0]
H [1]
S [1]
SUM [0, 1]
S [0]



(X**(r2 + s1 + s2)*Z**(r1 + r2 + s2))x(X**(r2 + s2)*Z**(r1 + r2 + s1))

I  ->  Z
Different Phase
X  ->  X
Different Phase
Z  ->  Y
Different Phase
Y  ->  I
Different Phase



